## **Contoh: Multi-AI Agen untuk Pemesanan Hotel**

Di dunia yang serba cepat saat ini, merencanakan perjalanan bisnis melibatkan lebih dari sekadar memesan tiket pesawat dan kamar hotel. Dibutuhkan tingkat koordinasi dan efisiensi yang sering kali sulit dicapai. Di sinilah Multi-AI Agen berperan, mengubah cara kita mengelola kebutuhan perjalanan.

Bayangkan memiliki tim agen pintar yang siap membantu Anda, bekerja sama untuk menangani setiap aspek perjalanan Anda dengan presisi dan kemudahan. Dengan teknologi AI canggih kami, kami telah menciptakan agen khusus untuk layanan pemesanan dan pengaturan jadwal perjalanan, memastikan pengalaman perjalanan yang lancar dan bebas stres.

Ini adalah skenario dasar. Saat merencanakan perjalanan bisnis, kita perlu berkonsultasi dengan agen perjalanan bisnis untuk mendapatkan informasi tiket pesawat, informasi hotel, dan sebagainya. Melalui AI Agen, kita dapat membangun agen untuk layanan pemesanan dan agen untuk pengaturan jadwal perjalanan agar dapat berkolaborasi dan meningkatkan tingkat kecerdasan.


# Inisialisasi Layanan Azure AI Agent dan Mendapatkan Informasi Konfigurasi dari **.env**

### **.env**

Buat file .env

**.env** berisi string koneksi untuk Layanan Azure AI Agent, model yang digunakan oleh AOAI, serta API, ENDPOINT, dan layanan Google API Search terkait lainnya.

- **AZURE_AI_AGENT_MODEL_DEPLOYMENT_NAME** = "Nama Deployment Model Layanan Azure AI Agent Anda"

[**NOTE**] Anda memerlukan model dengan Batas Laju 100.000 (Token per menit) dan Batas Laju 600 (Permintaan per menit).

  Anda dapat memperoleh model di Azure AI Foundry - Model dan Endpoint.

- **AZURE_AI_AGENT_PROJECT_CONNECTION_STRING** = "String Koneksi Proyek Layanan Azure AI Agent Anda"

  Anda dapat menemukan string koneksi proyek di bagian ikhtisar proyek Anda di Layar Portal AI Foundry.

- **SERPAPI_SEARCH_API_KEY** = "Kunci API Pencarian SERPAPI Anda"
- **SERPAPI_SEARCH_ENDPOINT** = "Endpoint Pencarian SERPAPI Anda"

Untuk mendapatkan Nama Deployment Model dan String Koneksi Proyek dari Layanan Azure AI Agent, Anda perlu membuat Layanan Azure AI Agent. Disarankan untuk menggunakan [template ini](https://portal.azure.com/#create/Microsoft.Template/uri/https%3A%2F%2Fraw.githubusercontent.com%2Ffosteramanda%2Fazure-agent-quickstart-templates%2Frefs%2Fheads%2Fmaster%2Fquickstarts%2Fmicrosoft.azure-ai-agent-service%2Fstandard-agent%2Fazuredeploy.json) untuk membuatnya secara langsung (***Catatan:*** Layanan Azure AI Agent saat ini hanya tersedia di wilayah tertentu. Disarankan untuk merujuk pada [tautan ini](https://learn.microsoft.com/en-us/azure/ai-services/agents/concepts/model-region-support) untuk mengatur wilayahnya).

Agen perlu mengakses SERPAPI. Disarankan untuk mendaftar menggunakan [tautan ini](https://serpapi.com/searches). Setelah mendaftar, Anda dapat memperoleh API KEY dan ENDPOINT unik.


# Masuk ke Azure

Sekarang Anda perlu masuk ke Azure. Buka terminal di VScode dan jalankan perintah `az login`.


# Pengaturan

Untuk menjalankan notebook ini, Anda perlu menginstal pustaka-pustaka berikut. Berikut adalah daftar pustaka yang diperlukan dan perintah pip install yang sesuai:

azure-identity: Untuk autentikasi Azure.  
requests: Untuk membuat permintaan HTTP.  
semantic-kernel: Untuk kerangka kerja semantic kernel (dengan asumsi ini adalah pustaka khusus atau spesifik, Anda mungkin perlu menginstalnya dari sumber atau repositori tertentu).  


In [ ]:
!pip install azure-identity
!pip install requests
!pip install semantic-kernel
!pip install --upgrade semantic_kernel
!pip install azure-cli

# Penjelasan:
import asyncio: Ini mengimpor modul asyncio, yang menyediakan dukungan untuk pemrograman asinkron di Python. Modul ini memungkinkan Anda menulis kode secara bersamaan menggunakan sintaks async dan await.
from typing import Annotated: Ini mengimpor tipe Annotated dari modul typing. Annotated digunakan untuk menambahkan metadata pada petunjuk tipe, yang dapat berguna untuk berbagai tujuan seperti validasi, dokumentasi, atau alat bantu.


In [ ]:
import asyncio,os
from typing import Annotated

# Penjelasan:
Dengan menggunakan from dotenv import load_dotenv dan load_dotenv(), Anda dapat dengan mudah mengelola pengaturan konfigurasi dan informasi sensitif (seperti kunci API dan URL database) dalam file .env, menjaga agar informasi tersebut terpisah dari kode sumber Anda, serta membuat aplikasi Anda lebih aman dan lebih mudah untuk dikonfigurasi.


In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Penjelasan:

Pernyataan Impor: from azure.identity.aio import DefaultAzureCredential: Pernyataan ini mengimpor kelas DefaultAzureCredential dari modul azure.identity.aio. Bagian aio pada nama modul menunjukkan bahwa modul ini dirancang untuk operasi asinkron.

Tujuan DefaultAzureCredential: Kelas DefaultAzureCredential adalah bagian dari Azure SDK untuk Python. Kelas ini menyediakan cara bawaan untuk melakukan autentikasi dengan layanan Azure. DefaultAzureCredential mencoba melakukan autentikasi menggunakan beberapa metode dalam urutan tertentu, seperti variabel lingkungan, managed identity, dan kredensial Azure CLI.

Operasi Asinkron: Modul aio menunjukkan bahwa kelas DefaultAzureCredential mendukung operasi asinkron. Ini berarti Anda dapat menggunakannya dengan asyncio untuk melakukan permintaan autentikasi tanpa memblokir.


In [ ]:
from azure.identity.aio import DefaultAzureCredential

# Penjelasan:
Mengimpor berbagai modul dan kelas dari paket semantic_kernel. Berikut adalah rincian dari setiap impor:

AgentGroupChat dari semantic_kernel.agents: Kelas ini menangani fungsi terkait obrolan grup untuk agen AI. AzureAIAgent dan AzureAIAgentSettings dari semantic_kernel.agents.azure_ai

AzureAIAgent: Kelas ini digunakan untuk membuat dan mengelola agen AI yang memanfaatkan layanan Azure AI.

AzureAIAgentSettings: Kelas ini digunakan untuk mengatur konfigurasi untuk AzureAIAgent. TerminationStrategy dari semantic_kernel.agents.strategies.termination.termination_strategy:

Kelas ini mendefinisikan strategi untuk menghentikan eksekusi agen AI di bawah kondisi tertentu. ChatMessageContent dari semantic_kernel.contents.chat_message_content:

Kelas ini digunakan untuk menangani konten pesan obrolan.
AuthorRole dari semantic_kernel.contents.utils.author_role:

Kelas ini mendefinisikan berbagai peran untuk penulis dalam konteks pesan obrolan.

kernel_function dari semantic_kernel.functions.kernel_function_decorator: Dekorator ini digunakan untuk mendefinisikan fungsi kernel, yaitu fungsi yang dapat dijalankan dalam kerangka kerja semantic kernel.
Impor ini menyiapkan komponen yang diperlukan untuk membuat dan mengelola agen AI yang dapat berinteraksi dalam lingkungan obrolan grup, kemungkinan untuk tugas seperti memesan hotel atau aktivitas serupa.


In [ ]:
from semantic_kernel.agents import AgentGroupChat
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentSettings
from semantic_kernel.agents.strategies.termination.termination_strategy import TerminationStrategy
from semantic_kernel.contents import ChatMessageContent
from semantic_kernel.contents import AuthorRole
from semantic_kernel.functions.kernel_function_decorator import kernel_function

# Penjelasan:
Selanjutnya, kita mengimpor kelas CodeInterpreterTool dari modul azure.ai.projects.models.

CodeInterpreterTool: Kelas ini adalah bagian dari Azure AI SDK dan digunakan untuk menginterpretasikan serta mengeksekusi kode dalam konteks proyek AI. Kelas ini menyediakan fungsi untuk menjalankan potongan kode, menganalisis kode, atau mengintegrasikan eksekusi kode dalam alur kerja AI.  
Impor ini menyiapkan komponen yang diperlukan untuk menggunakan CodeInterpreterTool dalam proyek Anda, yang dapat berguna untuk tugas-tugas yang melibatkan interpretasi dan eksekusi kode secara dinamis.


In [ ]:
from azure.ai.projects.models import CodeInterpreterTool

# Penjelasan: 
Kelas ApprovalTerminationStrategy menyediakan strategi khusus untuk menghentikan operasi agen AI. Agen akan berhenti jika pesan terakhir dalam riwayat interaksinya mengandung kata "saved". Ini bisa berguna dalam skenario di mana tugas agen dianggap selesai setelah menerima konfirmasi bahwa sesuatu telah "saved". Tentukan metode interaksi. Setelah rencana reservasi disimpan, agen dapat dihentikan ketika menerima sinyal "saved".


In [ ]:
class ApprovalTerminationStrategy(TerminationStrategy):
    """A strategy for determining when an agent should terminate."""

    async def should_agent_terminate(self, agent, history):
        """Check if the agent should terminate."""
        return "saved" in history[-1].content.lower()

# Penjelasan:

Baris kode tersebut menginisialisasi objek AzureAIAgentSettings dengan pengaturan default atau yang telah ditentukan sebelumnya dengan memanggil metode create(). Objek pengaturan ini (ai_agent_settings) kemudian dapat digunakan untuk mengonfigurasi dan mengelola instance AzureAIAgent.


In [ ]:
ai_agent_settings = AzureAIAgentSettings.create()

# Penjelasan:
Dengan mengimpor pustaka requests, Anda dapat dengan mudah membuat permintaan HTTP dan berinteraksi dengan layanan web dalam kode Python Anda.


In [ ]:
import requests

# Penjelasan:
Ini adalah variabel yang menyimpan kunci API untuk mengakses layanan API SERP (Search Engine Results Page). Kunci API adalah pengenal unik yang digunakan untuk mengautentikasi permintaan yang terkait dengan akun Anda.

'GOOGLE_SEARCH_API_KEY': Ini adalah string placeholder. Anda perlu mengganti 'GOOGLE_SEARCH_API_KEY' dengan kunci API SERP Anda yang sebenarnya.

Tujuan: Tujuan dari baris ini adalah untuk menyimpan kunci API dalam sebuah variabel sehingga dapat digunakan untuk mengautentikasi permintaan ke layanan API SERP. Kunci API diperlukan untuk mengakses layanan dan melakukan pencarian.

Cara Mendapatkan Kunci API SERP: Untuk mendapatkan kunci API SERP, ikuti langkah-langkah umum berikut di https://serpapi.com (langkah-langkah spesifik mungkin berbeda tergantung pada layanan API SERP yang Anda gunakan):

Pilih Layanan API SERP: Ada beberapa layanan API SERP yang tersedia, seperti SerpAPI, Google Custom Search JSON API, dan lainnya. Pilih yang paling sesuai dengan kebutuhan Anda.

Daftar untuk Akun:

Kunjungi situs web layanan API SERP yang dipilih https://www.serpapi.com dan daftar untuk sebuah akun. Anda mungkin perlu memberikan beberapa informasi dasar dan memverifikasi alamat email Anda.

Buat Kunci API:

Setelah mendaftar, masuk ke akun Anda dan navigasikan ke bagian API atau dashboard. Cari opsi untuk membuat atau menghasilkan kunci API baru.
Salin Kunci API:

Setelah kunci API dihasilkan, salin kunci tersebut. Kunci ini akan digunakan untuk mengautentikasi permintaan Anda ke layanan API SERP.
Ganti Placeholder:

Ganti placeholder di file .env Anda.


In [ ]:
SERPAPI_SEARCH_API_KEY=os.getenv('SERPAPI_SEARCH_API_KEY')

In [ ]:
SERPAPI_SEARCH_ENDPOINT = os.getenv('SERPAPI_SEARCH_ENDPOINT')

# Penjelasan:
Kelas BookingPlugin menyediakan metode untuk memesan hotel dan penerbangan menggunakan Serpapi.com Google Search API. Kelas ini membangun parameter yang diperlukan, mengirim permintaan API, dan memproses respons untuk mengembalikan informasi pemesanan yang relevan. Kunci API (SERPAPI_SEARCH_API_KEY) dan endpoint (SERPAPI_SEARCH_ENDPOINT) digunakan untuk mengautentikasi dan mengirim permintaan ke Google Search API.


In [ ]:
# Define Booking Plugin
class BookingPlugin:
    """Booking Plugin for customers"""
    @kernel_function(description="booking hotel")
    def booking_hotel(self,query: Annotated[str, "The name of the city"], check_in_date: Annotated[str, "Hotel Check-in Time"], check_out_date: Annotated[str, "Hotel Check-in Time"])-> Annotated[str, "Return the result of booking hotel infomation"]:

        params = {
            "engine": "google_hotels",
            "q": query,
            "check_in_date": check_in_date,
            "check_out_date": check_out_date,
            "adults": "2",
            "currency": "USD",
            "gl": "us",
            "hl": "en",
            "api_key": SERPAPI_SEARCH_API_KEY
        }

        response = requests.get(SERPAPI_SEARCH_ENDPOINT, params=params)
        if response.status_code == 200:
            response = response.json()
            return response["properties"]
        else:
            return None

    
    @kernel_function(description="booking fight")
    def  booking_fight(self,origin: Annotated[str, "The name of Departure"], destination: Annotated[str, "The name of Destination"], outbound_date: Annotated[str, "The date of outbound"], return_date: Annotated[str, "The date of Return_date"])-> Annotated[str, "Return the result of booking fight infomation"]:
        
        go_params = {
            "engine": "google_flights",   
            "departure_id": origin,
            "arrival_id": destination,
            "outbound_date": outbound_date,
            "return_date": return_date,  
            "currency": "USD",
            "hl": "en",
            "api_key": SERPAPI_SEARCH_API_KEY  
        }

        print(go_params)

        go_response = requests.get(SERPAPI_SEARCH_ENDPOINT, params=go_params)


        result = ''

        if go_response.status_code == 200:
            response = go_response.json()

            result += "# outbound \n " + str(response)
        else:
            print('error!!!')
            # return None

        
        back_params = {
            "engine": "google_flights",   
            "departure_id": destination,
            "arrival_id": origin,
            "outbound_date": return_date,
            "return_date": return_date,  
            "currency": "USD",
            "hl": "en",
            "api_key": SERPAPI_SEARCH_API_KEY  
        }


        print(back_params)


        back_response = requests.get(SERPAPI_SEARCH_ENDPOINT, params=back_params)



        if back_response.status_code == 200:
            response = back_response.json()

            result += "\n # return \n"  + str(response)

        else:
            print('error!!!')
            # return None
        
        print(result)

        return result

        


# Penjelasan:
Kelas SavePlugin menyediakan metode saving_plan untuk menyimpan rencana perjalanan menggunakan layanan Azure AI. Kelas ini mengatur kredensial Azure, membuat agen AI, memproses masukan pengguna untuk menghasilkan dan menyimpan konten rencana perjalanan, serta menangani operasi penyimpanan file dan pembersihan. Metode ini mengembalikan "Saved" setelah berhasil diselesaikan.


In [ ]:
class SavePlugin:
    """Save Plugin for customers"""
    @kernel_function(description="saving plan")
    async def saving_plan(self,tripplan: Annotated[str, "The content of trip plan"])-> Annotated[str, "Return status of save content"]:

        async with (
            DefaultAzureCredential() as creds,
            AzureAIAgent.create_client(
                credential=creds,
                conn_str=ai_agent_settings.project_connection_string.get_secret_value(),
            ) as client,
        ):

            code_interpreter = CodeInterpreterTool()
            
            agent_definition = await client.agents.create_agent(
                model=ai_agent_settings.model_deployment_name,
                tools=code_interpreter.definitions,
                tool_resources=code_interpreter.resources,
            )


            agent = AzureAIAgent(
                client=client,
                definition=agent_definition,
            )

            thread = await client.agents.create_thread()


            user_inputs = [
                """
            
                        You are my Python programming assistant. Generate code,save """+ tripplan +
                        
                    """    
                        and execute it according to the following requirements

                        1. Save blog content to trip-{YYMMDDHHMMSS}.md

                        2. give me the download this file link
                    """
            ]



            try:
                for user_input in user_inputs:
                    # Add the user input as a chat message
                    await agent.add_chat_message(
                        thread_id=thread.id, message=ChatMessageContent(role=AuthorRole.USER, content=user_input)
                    )
                    print(f"# User: '{user_input}'")
                    # Invoke the agent for the specified thread
                    async for content in agent.invoke(thread_id=thread.id):
                        if content.role != AuthorRole.TOOL:
                            print(f"# Agent: {content.content}")

                    
                    messages = await client.agents.list_messages(thread_id=thread.id)

                    # OpenAIPageableListOfThreadMessage
                    # OpenAIPageableListOfThreadMessage


                    for file_path_annotation in messages.file_path_annotations:

                            file_name = os.path.basename(file_path_annotation.text)

                            await client.agents.save_file(file_id=file_path_annotation.file_path.file_id, file_name=file_name,target_dir="./trip")

                    
            finally:
                await client.agents.delete_thread(thread.id)
                await client.agents.delete_agent(agent.id)


        return "Saved"

# Penjelasan:
Kode ini mengatur agen Azure AI untuk menangani pemesanan penerbangan dan hotel, serta menyimpan rencana perjalanan berdasarkan masukan pengguna. Kode ini menggunakan kredensial Azure untuk membuat dan mengonfigurasi agen, memproses masukan pengguna melalui obrolan grup, dan memastikan pembersihan yang tepat setelah tugas selesai. Agen-agen ini menggunakan plugin tertentu (BookingPlugin dan SavePlugin) untuk melaksanakan tugas masing-masing.


In [ ]:
async with (
    DefaultAzureCredential() as creds,
    AzureAIAgent.create_client(
        credential=creds,
        conn_str=ai_agent_settings.project_connection_string.get_secret_value(),
    ) as client,
):
    BOOKING_AGENT_NAME = "BookingAgent"
    BOOKING_AGENT_INSTRUCTIONS = """
    You are a booking agent. Help me book flights or hotels.

    Thought: Please understand the user's intention and confirm whether to use the reservation system to complete the task.

    Actions:
    - For flight bookings, convert the departure and destination names into airport codes.
    - Use the appropriate API for hotel or flight bookings. Verify that all necessary parameters are available. If any parameters are missing, ask the user to provide them. If all parameters are complete, call the corresponding function.
    - If the task is not related to hotel or flight booking, respond with the final answer only.
    - Output the results using a markdown table:
      - For flight bookings, output separate outbound and return contents in the order of:
        Departure Airport | Airline | Flight Number | Departure Time | Arrival Airport | Arrival Time | Duration | Airplane | Travel Class | Price (USD) | Legroom | Extensions | Carbon Emissions (kg).
      - For hotel bookings, output in the order of:
        Property Name | Property Description | Check-in Time | Check-out Time | Prices | Nearby Places | Hotel Class | GPS Coordinates.
    """

    SAVE_AGENT_NAME = "SaveAgent"
    SAVE_AGENT_INSTRUCTIONS = """
    You are a save tool agent. Help me to save the trip plan.
    """

    # Create agent definition
    booking_agent_definition = await client.agents.create_agent(
        model=ai_agent_settings.model_deployment_name,
        name=BOOKING_AGENT_NAME,
        instructions=BOOKING_AGENT_INSTRUCTIONS,
    )

    # Create the AzureAI Agent
    booking_agent = AzureAIAgent(
        client=client,
        definition=booking_agent_definition,
        # Optionally configure polling options
        # polling_options=RunPollingOptions(run_polling_interval=timedelta(seconds=1)),
    )

    # Add the sample plugin to the kernel
    booking_agent.kernel.add_plugin(BookingPlugin(), plugin_name="booking")

    # Create agent definition
    save_agent_definition = await client.agents.create_agent(
        model=ai_agent_settings.model_deployment_name,
        name=SAVE_AGENT_NAME,
        instructions=SAVE_AGENT_INSTRUCTIONS
    )

    # Create the AzureAI Agent
    save_agent = AzureAIAgent(
        client=client,
        definition=save_agent_definition,
    )

    save_agent.kernel.add_plugin(SavePlugin(), plugin_name="saving")

    user_inputs = [
        "I have a business trip from London to New York in Feb 20 2025 to Feb 27 2025 ,help me to book a hotel and fight tickets and save it"
    ]

    chat = AgentGroupChat(
        agents=[booking_agent, save_agent],
        termination_strategy=ApprovalTerminationStrategy(agents=[save_agent], maximum_iterations=10),
    )

    try:
        for user_input in user_inputs:
            # Add the user input as a chat message
            await chat.add_chat_message(
                ChatMessageContent(role=AuthorRole.USER, content=user_input)
            )
            print(f"# User: '{user_input}'")

            async for content in chat.invoke():
                print(f"# {content.role} - {content.name or '*'}: '{content.content}'")

            print(f"# IS COMPLETE: {chat.is_complete}")

            print("*" * 60)
            print("Chat History (In Descending Order):\n")
            async for message in chat.get_chat_messages(agent=save_agent):
                print(f"# {message.role} - {message.name or '*'}: '{message.content}'")
    finally:
        await chat.reset()
        await client.agents.delete_agent(save_agent.id)
        await client.agents.delete_agent(booking_agent.id)



---

**Penafian**:  
Dokumen ini telah diterjemahkan menggunakan layanan penerjemahan AI [Co-op Translator](https://github.com/Azure/co-op-translator). Meskipun kami berusaha untuk memberikan hasil yang akurat, harap diketahui bahwa terjemahan otomatis mungkin mengandung kesalahan atau ketidakakuratan. Dokumen asli dalam bahasa aslinya harus dianggap sebagai sumber yang otoritatif. Untuk informasi yang bersifat kritis, disarankan menggunakan jasa penerjemahan profesional oleh manusia. Kami tidak bertanggung jawab atas kesalahpahaman atau penafsiran yang keliru yang timbul dari penggunaan terjemahan ini.
